# Ансамблирование моделей

В этом ноутбуке рассмотрим примеры того, как можно собирать и обучать ансамбль моделей.

Начнем с подключения необходимых библиотек:

In [ ]:
import numpy as np
import torch
from torch.optim.lr_scheduler import ExponentialLR

import src.callbacks.curve
import src.callbacks.heatmap
import src.callbacks.progress
import src.callbacks.save
import src.problems
from src.callbacks.callbacks_organizer import EnsembleCallbacksOrganizer
from src.callbacks.heatmap import Grid
from src.generation.rectangle_generator import *
from src.neural_network import (
    FNN,
    ResNet,
    XavierFNN,
)
from src.neural_network.ensemble import (
    EnsembleTrainer,
    ensemble_builder,
)
from src.utils import set_device

Зафиксируем random_seed'ы и выберем ускоритель на котором будет обучаться наша модель. Если вам доступны GPU с поддержкой cuda, то именно этот девайс будет использован. Если вы хотите использовать CPU, используйте: 

`set_device('cpu')`

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

set_device()

В этом примере мы уже возьмем готовую сформулированную проблему: уравнение Навье-Стокса с блоком. **#TODO ссылка туда, где будет рассказано про проблемсы**

In [ ]:
conditions, input_dim, output_dim = src.problems.navier_stocks_equation_with_block()

Итак, приступим к сборке ансамбля моделей. В этом примере мы рассмотрим случай, когда предобученных моделей у нас нет и мы хотим обучить несколько моделей с нуля, а затем объединить их предсказания мета-моделью.

Используем 3 разных нейросети: `ResNet`, `FNN` и `XavierFNN`.

In [ ]:
models = [
    ResNet([input_dim, 32, 64, 64, 32, output_dim]),
    FNN([input_dim, 128, 256, 128, output_dim]),
    XavierFNN([input_dim, 128, 128, 128, 128, output_dim]),
]

Дальше мы можем организовать работу несколькими способами в зависимости от того, насколько мы хотим варьировать отдельные члены ансамбля. Ниже приведен пример, когда для каждой модели создаются одинаковые генераторы точек и оптимизаторы, но ничто не мешает сделать их разными для каждой модели.

In [ ]:
generators_domain = [
    UniformGeneratorRect(n_points=5000, method="uniform") for _ in range(3)
]

generators_boundary = [
    UniformGeneratorRect(n_points=500, method="uniform") for _ in range(3)
]

optimizers = [torch.optim.Adam(model.parameters()) for model in models]

schedulers = [ExponentialLR(optimizer=opt, gamma=0.999) for opt in optimizers]

После того, как мы собрали все части `EnsembleInstance`, можем собрать конфинг для ансамбля:

In [ ]:
ensemble_config = ensemble_builder(
    models,
    generators_domain,
    generators_boundary,
    [0, 1, 2, 3],
    optimizers,
    schedulers,
    conditions,
)

Далее, мы используем различные коллбеки для отрисовки решений и визуализации процесса обучения. В примере ниже мы будем создавать коллбеки которые используют `Grid`. Создадим один объект `Grid` для всех таких коллбеков. Если необходима отризовка на разных сетках, можно также передавать разные объекты.

Укажем также период вызова коллбеков и папку, в которую будем сохранять результаты эксперимента.

In [ ]:
PERIOD = 200
gird = Grid.from_pinn(ensemble_config[0].pinn, 15000)
SAVE_DIR = "ensemble_experiment"

В нашей задачи мы имеем выход размерности 3: ($u, v, p$). Чтобы сделать визуализацию каждого поля отдельно, создадим нужные коллбеки, передав соответствующие `output_index`.

In [ ]:
callbacks = [
    src.callbacks.progress.TqdmBar("Epoch {epoch} lr={lr:.2e} Total={total_loss:.2e}"),
    src.callbacks.curve.LearningRateCurve(SAVE_DIR, PERIOD, log_scale=False),
    src.callbacks.curve.LossCurve(SAVE_DIR, PERIOD),
    src.callbacks.curve.GridResidualCurve(
        SAVE_DIR,
        PERIOD,
        grid=gird,
    ),
    src.callbacks.heatmap.HeatmapPrediction(
        SAVE_DIR, PERIOD, grid=gird, save_mode="html", output_index=0
    ),
    src.callbacks.heatmap.HeatmapPrediction(
        SAVE_DIR,
        PERIOD,
        grid=gird,
        save_mode="html",
        output_index=1,
    ),
    src.callbacks.heatmap.HeatmapPrediction(
        SAVE_DIR, PERIOD, grid=gird, save_mode="html", output_index=2
    ),
]

Почти готово! Осталось обернуть коллбеки в специальный класс `EnsembleCallbacksOrganizer` и создать `EnsembleTrainer`. Нужно использовать именно эти классы, вместо обычных `CallbacksOrganizer` и `Trainer`, т.к. работа с таким ансамблем требует некоторых дополнительных методов. 

In [ ]:
callbacks_orgaziner = EnsembleCallbacksOrganizer(callbacks)

trainer = EnsembleTrainer(
    ensemble_config,
    callbacks_organizer=callbacks_orgaziner,
    num_epochs=1000,
    output_dim=output_dim,
)

trainer.train()